In [7]:
import pandas as pd
import os

#path
flights_dir = "/home/sagemaker-user/ADS-508-Project/Raw_data/raw_on-time/"

# List all CSV files in the directory
csv_files = [f for f in os.listdir(flights_dir) if f.endswith(".csv")]

# Load and concatenate all CSVs into one DataFrame
flight_dfs = [pd.read_csv(os.path.join(flights_dir, file)) for file in csv_files]
san_ot_df = pd.concat(flight_dfs, ignore_index=True)

# Display first few rows
san_ot_df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,CANCELLED,CANCELLATION_CODE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DEP_DELAY
0,4/1/2023 12:00:00 AM,AA,1,JFK,LAX,725,722.0,0.0,-1.0,0700-0759,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/1/2023 12:00:00 AM,AA,10,LAX,JFK,2215,959.0,1.0,12.0,2200-2259,0.0,NaN,689.0,0.0,0.0,0.0,0.0,NaN
2,4/1/2023 12:00:00 AM,AA,1009,DFW,SBP,1017,1015.0,0.0,-1.0,1000-1059,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/1/2023 12:00:00 AM,AA,1009,SBP,DFW,1235,1229.0,0.0,-1.0,1200-1259,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/1/2023 12:00:00 AM,AA,1017,DFW,SBP,1913,1908.0,0.0,-1.0,1900-1959,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import pandas as pd
import os

# Define the directory path
weather_dir = "/home/sagemaker-user/ADS-508-Project/Raw_data/raw_weather/"

# List all weather CSV files
weather_files = [f for f in os.listdir(weather_dir) if f.endswith(".csv")]

# Load and concatenate all weather data
weather_dfs = [
    pd.read_csv(os.path.join(weather_dir, file), dtype=str, low_memory=False)
    for file in weather_files
]
metar_df = pd.concat(weather_dfs, ignore_index=True)

# Display first few rows
metar_df.head()

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,OC1,OD1,OE1,OE2,OE3,RH1,RH2,RH3,REM,EQD
0,72290023188,2023-01-01T00:00:00,4,32.7336,-117.1831,4.6,"SAN DIEGO INTERNATIONAL AIRPORT, CA US",FM-12,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYN08672290 12366 8//06 10161 20128 30148 4016...,NaN
1,72290023188,2023-01-01T00:10:00,7,32.7336,-117.1831,4.6,"SAN DIEGO INTERNATIONAL AIRPORT, CA US",FM-16,KSAN,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10512/31/22 16:10:03 SPECI KSAN 010010Z 180...,NaN
2,72290023188,2023-01-01T00:51:00,7,32.7336,-117.1831,4.6,"SAN DIEGO INTERNATIONAL AIRPORT, CA US",FM-15,KSAN,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET12712/31/22 16:51:03 METAR KSAN 010051Z 160...,NaN
3,72290023188,2023-01-01T01:51:00,7,32.7336,-117.1831,4.6,"SAN DIEGO INTERNATIONAL AIRPORT, CA US",FM-15,KSAN,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET12612/31/22 17:51:03 METAR KSAN 010151Z 150...,NaN
4,72290023188,2023-01-01T02:16:00,7,32.7336,-117.1831,4.6,"SAN DIEGO INTERNATIONAL AIRPORT, CA US",FM-16,KSAN,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10512/31/22 18:16:03 SPECI KSAN 010216Z 140...,NaN


In [12]:
import boto3

# Define S3 bucket and file paths
bucket_name = "sagemaker-us-east-1-230393891640"  # Your confirmed S3 bucket
clean_flights_path = "/home/sagemaker-user/ADS-508-Project/clean_data/SAN_OT_report.csv"
clean_weather_path = "/home/sagemaker-user/ADS-508-Project/clean_data/organized_metar_data.csv"

# Create S3 client
s3 = boto3.client("s3")

# Upload cleaned data to S3
s3.upload_file(clean_flights_path, bucket_name, "clean_data/SAN_OT_report.csv")
s3.upload_file(clean_weather_path, bucket_name, "clean_data/organized_metar_data.csv")

print("✅ Cleaned data successfully uploaded to S3.")

✅ Cleaned data successfully uploaded to S3.


In [13]:
# List files in the S3 bucket to confirm upload
response = s3.list_objects_v2(Bucket=bucket_name, Prefix="clean_data/")

# Print uploaded file names
print("Files in S3:")
for obj in response.get("Contents", []):
    print(obj["Key"])

Files in S3:
clean_data/SAN_OT_report.csv
clean_data/organized_metar_data.csv


In [15]:
import boto3

# Define S3 bucket
bucket_name = "sagemaker-us-east-1-230393891640"  # Your confirmed S3 bucket

# List files in S3
s3 = boto3.client("s3")
response = s3.list_objects_v2(Bucket=bucket_name, Prefix="clean_data/")

print("Files in S3:")
for obj in response.get("Contents", []):
    print(obj["Key"])

Files in S3:
clean_data/SAN_OT_report.csv
clean_data/organized_metar_data.csv


In [19]:
import boto3

# Define S3 bucket name
bucket_name = "sagemaker-us-east-1-230393891640"

#  file paths in project directory
local_flight_path = "/home/sagemaker-user/ADS-508-Project/clean_data/SAN_OT_report.csv"
local_weather_path = "/home/sagemaker-user/ADS-508-Project/clean_data/organized_metar_data.csv"

# Define S3 file paths
s3_flight_path = "clean_data/SAN_OT_report.csv"
s3_weather_path = "clean_data/organized_metar_data.csv"

# Create an S3 client
s3 = boto3.client("s3")

# Download files from S3 into the correct local directory
try:
    s3.download_file(bucket_name, s3_flight_path, local_flight_path)
    print(f"✅ {s3_flight_path} downloaded successfully to {local_flight_path}.")
except Exception as e:
    print(f"❌ Failed to download {s3_flight_path}: {e}")

try:
    s3.download_file(bucket_name, s3_weather_path, local_weather_path)
    print(f"✅ {s3_weather_path} downloaded successfully to {local_weather_path}.")
except Exception as e:
    print(f"❌ Failed to download {s3_weather_path}: {e}")

✅ clean_data/SAN_OT_report.csv downloaded successfully to /home/sagemaker-user/ADS-508-Project/clean_data/SAN_OT_report.csv.
✅ clean_data/organized_metar_data.csv downloaded successfully to /home/sagemaker-user/ADS-508-Project/clean_data/organized_metar_data.csv.


In [20]:
import pandas as pd

# Define file paths
flights_path = "/home/sagemaker-user/ADS-508-Project/clean_data/SAN_OT_report.csv"
weather_path = "/home/sagemaker-user/ADS-508-Project/clean_data/organized_metar_data.csv"

# Load datasets into Pandas
san_ot_df = pd.read_csv(flights_path)
metar_df = pd.read_csv(weather_path)

# Display first few rows
print("Flight Data Sample:")
display(san_ot_df.head())

print("\nWeather Data Sample:")
display(metar_df.head())

Flight Data Sample:


,FL_DATE,CRS_DEP_TIME,DEP_TIME,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,DEP_DEL15,CANCELLED,CANCELLATION_CODE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2023-01-01,07:15:00,07:13:00,AA,1055,SAN,DFW,0,0,NaN,0,0,0,0,0
1,2023-01-01,06:15:00,06:44:00,AA,1651,SAN,CLT,1,0,NaN,0,0,0,0,0
2,2023-01-01,22:59:00,22:50:00,AA,1663,SAN,ORD,0,0,NaN,0,0,0,0,0
3,2023-01-01,08:15:00,08:10:00,AA,1765,SAN,DFW,0,0,NaN,0,0,0,0,0
4,2023-01-01,14:50:00,14:47:00,AA,1939,SAN,DFW,0,0,NaN,0,0,0,0,0



Weather Data Sample:


,local_date,local_time,station_id,wind_dir_degrees,wind_speed_kt,wind_gust_kt,visibility_statute_mi,temperature_c,dewpoint_c,altimeter_hpa
0,12/31/22,16:10:03,KSAN,180.0,4.0,0.0,10.0,15.6,12.8,30.01
1,12/31/22,16:51:03,KSAN,160.0,6.0,0.0,8.0,15.6,13.3,30.00
2,12/31/22,17:51:03,KSAN,150.0,6.0,0.0,10.0,15.6,12.8,29.98
3,12/31/22,18:16:03,KSAN,140.0,10.0,0.0,10.0,15.6,12.8,29.97
4,12/31/22,18:38:03,KSAN,140.0,10.0,21.0,10.0,15.6,12.8,29.96


In [22]:
# Check column data types
print("Flight Data Types:")
print(san_ot_df.dtypes)

print("\nWeather Data Types:")
print(metar_df.dtypes)

# Check for missing values
print("\nMissing Values in Flight Data:")
print(san_ot_df.isnull().sum())

print("\nMissing Values in Weather Data:")
print(metar_df.isnull().sum())

Flight Data Types:
FL_DATE                object
CRS_DEP_TIME           object
DEP_TIME               object
OP_UNIQUE_CARRIER      object
OP_CARRIER_FL_NUM       int64
ORIGIN                 object
DEST                   object
DEP_DEL15               int64
CANCELLED               int64
CANCELLATION_CODE      object
CARRIER_DELAY           int64
WEATHER_DELAY           int64
NAS_DELAY               int64
SECURITY_DELAY          int64
LATE_AIRCRAFT_DELAY     int64
dtype: object

Weather Data Types:
local_date                object
local_time                object
station_id                object
wind_dir_degrees         float64
wind_speed_kt            float64
wind_gust_kt             float64
visibility_statute_mi    float64
temperature_c            float64
dewpoint_c               float64
altimeter_hpa            float64
dtype: object

Missing Values in Flight Data:
FL_DATE                     0
CRS_DEP_TIME                0
DEP_TIME                 2052
OP_UNIQUE_CARRIER           0


## Check if There is Potential Bias in the Dataset

In [23]:
# Check the distribution of delays
if 'DEP_DEL15' in san_ot_df.columns:
    print("\nFlight Delay Distribution:")
    print(san_ot_df["DEP_DEL15"].value_counts(normalize=True) * 100)
else:
    print("\nWarning: DEP_DEL15 column not found in flight data!")


Flight Delay Distribution:
DEP_DEL15
0    90.473599
1     9.526401
Name: proportion, dtype: float64


##  Undersampling (Reducing Majority Class)
### We balanced the dataset by downsampling the majority class(on-time flights).

In [24]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = san_ot_df[san_ot_df["DEP_DEL15"] == 0]
df_minority = san_ot_df[san_ot_df["DEP_DEL15"] == 1]

# Downsample majority class (match minority class count)
df_majority_downsampled = resample(df_majority, 
                                   replace=False,  # Sample without replacement
                                   n_samples=len(df_minority),  # Match minority class count
                                   random_state=42) 

# Combine minority class with downsampled majority class
balanced_df = pd.concat([df_majority_downsampled, df_minority])

# Check new class distribution
print("Balanced Flight Delay Distribution:")
print(balanced_df["DEP_DEL15"].value_counts(normalize=True) * 100)

Balanced Flight Delay Distribution:
DEP_DEL15
0    50.0
1    50.0
Name: proportion, dtype: float64
